In [15]:
import pandas as pd

data = pd.read_csv('Tweets_cleaned_data.csv')

In [16]:
data.shape

(1600000, 12)

In [17]:
data = data.sample(frac = 0.2)
data

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1
66659,0,1691902118,Sun May 03 18:51:20 PDT 2009,NO_QUERY,cimtrbl2,still sick n very bored lol any suggestions on...,still sick n very bored lol any suggestions on...,still sick n very any on what on,NaN,"['still', 'sick', 'n', 'very', 'any', 'on', 'w...","['still', 'sick', 'n']",3
1042629,4,1957227096,Thu May 28 23:52:59 PDT 2009,NO_QUERY,DarianLovesYou,ii think i'm overdosing on sprite lawl,ii think im overdosing on sprite lawl,think on sprite,NaN,"['think', 'on', 'sprite']","['think', 'sprite']",2
1380918,4,2052294910,Sat Jun 06 00:15:19 PDT 2009,NO_QUERY,jamiehitchcock,@dmcclure really sorry to hear that. hope he w...,really sorry to hear that hope he will feel be...,really sorry to hear that hope he will feel be...,NaN,"['really', 'sorry', 'to', 'hear', 'that', 'hop...","['really', 'sorry', 'hear', 'hope', 'feel', 'b...",7
101149,0,1794315683,Thu May 14 05:31:15 PDT 2009,NO_QUERY,SoozyJ,@themakelounge the link doesn't work i want t...,the link doesnt work i want to decorate cupcak...,the link doesnt work i want to decorate proper...,NaN,"['the', 'link', 'doesnt', 'work', 'i', 'want',...","['link', 'doesnt', 'work', 'want', 'decorate',...",7
1019882,4,1882268537,Fri May 22 06:33:32 PDT 2009,NO_QUERY,Scalper68,@g2trading ty,ty,NaN,NaN,[],[],0
...,...,...,...,...,...,...,...,...,...,...,...,...
1347261,4,2044500919,Fri Jun 05 09:27:03 PDT 2009,NO_QUERY,chantellmarie,@cacaubrazil o ok. i got it! spanish does it k...,o ok i got it spanish does it kinda the same a...,o i got it does it the same as for and os for ...,NaN,"['o', 'i', 'got', 'it', 'does', 'it', 'the', '...","['got', 'os', 'day', 'going']",4
1553382,4,2184624413,Mon Jun 15 16:10:49 PDT 2009,NO_QUERY,xocarissajonas,http://www.myspace.com/pop2kpromotions &lt;-- ...,lt wont stop tweeting about it untill ig et ad...,wont stop about it untill from its brand new,NaN,"['wont', 'stop', 'about', 'it', 'untill', 'fro...","['wont', 'stop', 'untill', 'brand', 'new']",5
1370333,4,2050994869,Fri Jun 05 19:40:12 PDT 2009,NO_QUERY,ahhingtina,@agelufailau hey it's hella funny! your new pi...,hey its hella funny your new pic hella looks l...,hey its funny your new pic like like always w ...,NaN,"['hey', 'its', 'funny', 'your', 'new', 'pic', ...","['hey', 'funny', 'new', 'pic', 'like', 'like',...",9
69008,0,1693111418,Sun May 03 21:35:40 PDT 2009,NO_QUERY,mosesmonster,@staceybeeeee north carolina! you moving to s...,north carolina you moving to santa monica this...,north you moving to this fall,NaN,"['north', 'you', 'moving', 'to', 'this', 'fall']","['north', 'moving', 'fall']",3


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

# parameter setting
n_samples = data.shape[0]
n_features = 1000
n_components = 20
n_top_words = 10

In [26]:
tf_vectorizer = CountVectorizer(max_df=0.90, min_df=2,
                                max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(data['text3'].values.astype('U'))
feature_name = tf_vectorizer.get_feature_names()
index_dict = dict(enumerate(feature_name))

In [27]:
tf

<320000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 1081249 stored elements in Compressed Sparse Row format>

In [28]:
print(index_dict)
print(data.iloc[4]['text3'])
print(tf.toarray()[4])

{0: 'able', 1: 'absolutely', 2: 'account', 3: 'ache', 4: 'actually', 5: 'add', 6: 'added', 7: 'adorable', 8: 'advice', 9: 'afford', 10: 'afraid', 11: 'afternoon', 12: 'age', 13: 'ago', 14: 'agree', 15: 'ah', 16: 'ahead', 17: 'aint', 18: 'air', 19: 'airport', 20: 'album', 21: 'alive', 22: 'alright', 23: 'amazing', 24: 'angry', 25: 'annoying', 26: 'answer', 27: 'anyways', 28: 'apparently', 29: 'apple', 30: 'appreciate', 31: 'area', 32: 'arent', 33: 'arm', 34: 'art', 35: 'article', 36: 'ask', 37: 'asleep', 38: 'ass', 39: 'ate', 40: 'august', 41: 'available', 42: 'aw', 43: 'awake', 44: 'away', 45: 'awesome', 46: 'awful', 47: 'babe', 48: 'baby', 49: 'background', 50: 'bad', 51: 'badly', 52: 'bag', 53: 'ball', 54: 'band', 55: 'bank', 56: 'bar', 57: 'barely', 58: 'bath', 59: 'battery', 60: 'beach', 61: 'beat', 62: 'beautiful', 63: 'bed', 64: 'beer', 65: 'believe', 66: 'best', 67: 'bet', 68: 'better', 69: 'big', 70: 'bike', 71: 'bird', 72: 'birthday', 73: 'bit', 74: 'bitch', 75: 'black', 76: '

In [29]:
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, n_components=20, random_state=0)

In [30]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [31]:
tf_feature_names = tf_vectorizer.get_feature_names()

In [32]:
print_top_words(lda, tf_feature_names, 10) #change n_top_words

Topic #0: just got ready pretty party car stop enjoy pic ago
Topic #1: need morning sick feeling wont havent girl baby leave family
Topic #2: dont know sad wait rain start waiting eat stupid money
Topic #3: twitter didnt nice people old talk mean picture heart concert
Topic #4: want right bad did come cool wow job stay idea
Topic #5: work think tonight nan doesnt look later god hour crazy
Topic #6: going night days hey amazing ya end needs hit goes
Topic #7: thanks tomorrow yeah bed bit play said lot check sweet
Topic #8: today watching working tired watch let cold woke food room
Topic #9: sleep getting little saw yesterday till hair use cute thinking
Topic #10: happy great thank say maybe trying follow stuff id read
Topic #11: hope sorry better soon live hear making music beautiful exam
Topic #12: oh thats hate house early big hi cause following course
Topic #13: make sure thing away thought hot weather funny outside place
Topic #14: day time new week school today doing sun ugh hell
T

In [2]:
import pandas as pd
aim = pd.read_csv("Tweets_sentiment.csv")
aim

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags,sentiment_score
0,0,2059014670,Sat Jun 06 16:04:41 PDT 2009,NO_QUERY,patriciya,@pearl sayang di tayo blockmates. omg the cas...,sayang di tayo blockmates omg the cases are so...,di the are so,[],"['di', 'the', 'are', 'so']",['di'],1,0,3.0
1,4,1548317796,Fri Apr 17 20:37:23 PDT 2009,NO_QUERY,emma__watson,in... scotland twitters may be sparse for a w...,in scotland twitters may be sparse for a while...,in may be sparse for a while but i intend to d...,[],"['in', 'may', 'be', 'sparse', 'for', 'a', 'whi...","['may', 'sparse', 'intend', 'best']",4,0,3.0
2,0,1998735976,Mon Jun 01 19:02:28 PDT 2009,NO_QUERY,PinkTribble,why has the sciatic discomfort chosen to retur...,why has the sciatic discomfort chosen to retur...,why the sciatic discomfort chosen to return ju...,[],"['why', 'the', 'sciatic', 'discomfort', 'chose...","['sciatic', 'discomfort', 'chosen', 'return', ...",8,0,1.0
3,4,2063144198,Sun Jun 07 01:31:48 PDT 2009,NO_QUERY,Raycast,"@pembsdave lol, sounds like a good way to be! ...",lol sounds like a good way to be enjoy your no...,like a good way to be enjoy your no,[],"['like', 'a', 'good', 'way', 'to', 'be', 'enjo...","['like', 'good', 'way', 'enjoy']",4,0,4.0
4,0,1992040864,Mon Jun 01 08:05:55 PDT 2009,NO_QUERY,laurennicole514,about to go to work.,about to go to work,about to go to work,[],"['about', 'to', 'go', 'to', 'work']","['go', 'work']",2,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316640,0,1881534122,Fri May 22 04:55:25 PDT 2009,NO_QUERY,gqgoat,"the rain still persists here in gainesville, f...",the rain still persists here in gainesville fl...,the rain still here in hope not too many to ce...,[],"['the', 'rain', 'still', 'here', 'in', 'hope',...","['rain', 'still', 'hope', 'many', 'central', '...",9,0,2.0
316641,0,2190311283,Tue Jun 16 02:21:51 PDT 2009,NO_QUERY,starchild20,@stacijshelton thanks -i am looking for inspoi...,thanks i am looking for inspoiration and encou...,thanks i am looking for and encouragement at m...,[],"['thanks', 'i', 'am', 'looking', 'for', 'and',...","['thanks', 'looking', 'encouragement', 'mo', '...",5,0,5.0
316642,4,2187459719,Mon Jun 15 20:18:48 PDT 2009,NO_QUERY,ryking,"@rickydee55 i do have a reputation, don't i? s...",i do have a reputation dont i sorry,i do have a reputation dont i sorry,[],"['i', 'do', 'have', 'a', 'reputation', 'dont',...","['reputation', 'dont', 'sorry']",3,0,3.0
316643,0,2175490828,Mon Jun 15 01:08:06 PDT 2009,NO_QUERY,Vivicam,emma you tool! and to think i had to walk home...,emma you tool and to think i had to walk home ...,emma you tool and to think i had to walk home ...,[],"['emma', 'you', 'tool', 'and', 'to', 'think', ...","['emma', 'tool', 'think', 'walk', 'home', 'dar...",9,0,5.0


In [3]:
pd.crosstab(aim['target'], aim['sentiment_score'], margins=True)

sentiment_score,1.0,2.0,3.0,4.0,5.0,All
target,,,,,,
0,62560,21484,31130,10036,33873,159083
4,23914,7128,26122,21683,78715,157562
All,86474,28612,57252,31719,112588,316645
